#Pix2Text

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!pip install -qU langchain jq tiktoken langchain_community langchain_chroma langchain-huggingface huggingface-hub sentence_transformers chromadb langchainhub
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Langchain

In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings, HuggingFaceHubEmbeddings, HuggingFaceHubEmbeddings
from langchain_huggingface import HuggingFacePipeline

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
import bs4, getpass, os, tiktoken
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate
from transformers import BitsAndBytesConfig

# Load, chunk and index the contents of the blog.
#loader = TextLoader("/content/output/output.md")
#docs = loader.load()

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
#num_tokens_from_string(question, "cl100k_base")
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
#similarity = cosine_similarity(query_result, document_result)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = Chroma(persist_directory="/content/drive/MyDrive/UWA/Sem 4/Capstone/Project/vector",embedding_function=hf)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:

#def format_docs(docs):
#    return "\n\n".join(" page number : "+ str(doc.metadata['page']) +" "+ doc.page_content for doc in docs)


# Function to format documents for inclusion in the context
def format_docs(docs):
    """
    Formats a list of documents into a string with content and metadata.

    Args:
        docs (list): A list of document objects with 'page_content' and 'metadata' attributes.

    Returns:
        str: A formatted string with each document's content and metadata.
    """
    return "\n\n".join(doc.page_content for doc in docs)

In [15]:

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline, BitsAndBytesConfig

model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2").to("cuda")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

changing the format docs function to include the page number as part of the content for each chunk, not only in the metadata.

Earlier we were using phi 3 mini which couldnt use complex prompts involving page number. after switching to phi 3 medium with flash attention 2, it can understand the prompt template better and extract page nmber and chapter number from chunks.

In [16]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000,device="cuda:0")
llm = HuggingFacePipeline(pipeline=pipe)

In [17]:
question = "What Ideal gas law?"

In [18]:
docs = retriever.get_relevant_documents(question)
total_tokens = 0
for doc in docs:
    # Extract the page content
    content = doc.page_content

    # Calculate the number of tokens in the page content
    tokens = num_tokens_from_string(content, "cl100k_base")
    total_tokens += tokens

print(total_tokens)

984


In [12]:
docs

[Document(metadata={'page': 18}, page_content='$$\nz={\\frac{p V} {n R T}}={\\frac{p} {\\rho_{M} R T}}=1 \\qquad\\qquad\\qquad\\qquad( 1. 5 6 )\n$$\n\nThis equation clearly is consistent with Eq. (1.11), which contains the definition of the absolute temperature. The limiting process indicated in Eq. (1.11) rigorously eliminates the virial coefficients to provide a precise definition; Eq. (1.56) covers a useful range of densities for practical calculations and is called the ideal gas law.\n\n## Partial pressures\n\nA useful quantity for dealing with the individual components in a gas mixture is partial pressure. The partial pressure of a component in a mixture, for example, component $A$ , is defined by the equation\n\n$$\np_{A} \\equiv P y_{A} \\qquad\\qquad\\qquad\\qquad\\qquad\\qquad( 1. 5 7 )\n$$\n\nP2E盟区CcR0m $A$ in mixture $y_{A}$ = mole fraction of component $A$ in mixture P = total pressure on mixture\n\nfalthearia resuresfor gien itue re aded, theresut i\n\n$$\np_{A}+p_{B}+p_{C

In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(question)

"Human: Answer the question based only on the following context:\n$$\nz={\\frac{p V} {n R T}}={\\frac{p} {\\rho_{M} R T}}=1 \\qquad\\qquad\\qquad\\qquad( 1. 5 6 )\n$$\n\nThis equation clearly is consistent with Eq. (1.11), which contains the definition of the absolute temperature. The limiting process indicated in Eq. (1.11) rigorously eliminates the virial coefficients to provide a precise definition; Eq. (1.56) covers a useful range of densities for practical calculations and is called the ideal gas law.\n\n## Partial pressures\n\nA useful quantity for dealing with the individual components in a gas mixture is partial pressure. The partial pressure of a component in a mixture, for example, component $A$ , is defined by the equation\n\n$$\np_{A} \\equiv P y_{A} \\qquad\\qquad\\qquad\\qquad\\qquad\\qquad( 1. 5 7 )\n$$\n\nP2E盟区CcR0m $A$ in mixture $y_{A}$ = mole fraction of component $A$ in mixture P = total pressure on mixture\n\nfalthearia resuresfor gien itue re aded, theresut i\n\n$

Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

Question:
What Ideal gas law?

3

NOTE FOR FAILURE:
template = """Answer the question with the specific page numbers where the relevant information is found, based on the provided context. All equations should be formatted in LaTeX.

The output was generated the questions

    Context:
    {context}

    Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

    Question:
    {question}

    """
    
prompt = ChatPromptTemplate.from_template(template)
Response should be well-organized and formatted for easy readability, including clear identification of all relevant page numbers.

Question:
What Ideal gas law?

1.56

What is the definition of the absolute temperature?

1.11

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56

What is the equation for the molar density?

1.55

What is the equation for the ideal gas law?

1.56

What is the equation for the partial pressure of a component in a gas mixture?

1.57

What is the equation for the compressibility factor?

1.56